## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 14s 0us/step
(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [20]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3),input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.00))

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,kernel_size=(3,3),input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))
classifier.add(MaxPooling2D(pool_size=(2,2)))
#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

C:\Users\aaron\Anaconda3\envs\cvclass\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\Users\aaron\Anaconda3\envs\cvclass\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 10s 192us/step - loss: 1.3458 - accuracy: 0.5295
Epoch 2/100
50000/50000 [==============================] - 6s 121us/step - loss: 0.9317 - accuracy: 0.6721
Epoch 3/100
50000/50000 [==============================] - 6s 116us/step - loss: 0.7614 - accuracy: 0.7318
Epoch 4/100
50000/50000 [==============================] - 6s 112us/step - loss: 0.6230 - accuracy: 0.7824
Epoch 5/100
50000/50000 [==============================] - 6s 112us/step - loss: 0.4943 - accuracy: 0.8262
Epoch 6/100
50000/50000 [==============================] - 6s 113us/step - loss: 0.3785 - accuracy: 0.8678
Epoch 7/100
50000/50000 [==============================] - 6s 112us/step - loss: 0.2794 - accuracy: 0.9022
Epoch 8/100
50000/50000 [==============================] - 6s 113us/step - loss: 0.2007 - accuracy: 0.9311
Epoch 9/100
50000/50000 [==============================] - 6s 113us/step - loss: 0.1646 - accuracy: 0.9430
Epoch 10/100
50000/50000 [==========

50000/50000 [==============================] - 6s 113us/step - loss: 0.0172 - accuracy: 0.9946
Epoch 77/100
50000/50000 [==============================] - 6s 113us/step - loss: 0.0224 - accuracy: 0.9933
Epoch 78/100
50000/50000 [==============================] - 6s 113us/step - loss: 0.0236 - accuracy: 0.9930
Epoch 79/100
50000/50000 [==============================] - 6s 113us/step - loss: 0.0146 - accuracy: 0.9957
Epoch 80/100
50000/50000 [==============================] - 6s 113us/step - loss: 0.0072 - accuracy: 0.9977
Epoch 81/100
50000/50000 [==============================] - 6s 117us/step - loss: 0.0139 - accuracy: 0.9960
Epoch 82/100
50000/50000 [==============================] - 6s 119us/step - loss: 0.0212 - accuracy: 0.9939
Epoch 83/100
50000/50000 [==============================] - 6s 114us/step - loss: 0.0145 - accuracy: 0.9957
Epoch 84/100
50000/50000 [==============================] - 6s 115us/step - loss: 0.0141 - accuracy: 0.9957
Epoch 85/100
50000/50000 [===============

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [21]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.2226470e-32, 0.0000000e+00, 3.6684389e-23, 1.0000000e+00,
        6.3203396e-15, 0.0000000e+00, 1.4885397e-32, 6.6325012e-32,
        1.6027312e-32, 0.0000000e+00]], dtype=float32)